In [1]:
# Create ClickUp tasks from PlusPortals

## PlusPortals Init Section

from logging import error
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import json
import requests
import calendar, time
from datetime import date, datetime, timedelta

# Try to create a ChromeDriver instance without installing
try:
    driver = webdriver.Chrome()
except:
    # If ChromeDriver is not found, install it and then create the instance
    ChromeDriverManager().install()
    driver = webdriver.Chrome()

wait = WebDriverWait(driver, 20)
longwait = WebDriverWait(driver, 60)

## ClickUp Init Section

# Load credentials from JSON configuration file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

username  = config.get("username")
password  = config.get("password")
api_token = config.get("api") # ClickUp API token

# ClickUp Space ID and List ID 
space_id = '9006014177'
list_id = '901100521675'

# API endpoint URLs
base_url = "https://api.clickup.com/api/v2"
tasks_url = f"{base_url}/list/{list_id}/task"

# Headers with authentication token
headers = {
    "Authorization": api_token,
    "Content-Type": "application/json"
}

def login():
    # Open the login page
    login_url = 'https://plusportals.com/springcreekacademy'
    driver.get(login_url)

    ## Enter the username and password
    driver.find_element(By.ID, "UserName").send_keys(username)
    driver.find_element(By.ID, "Password").send_keys(password)

    # Locate and click the "Sign in" button
    submit_txt = 'input[type="submit"][value="Sign in"]'
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, submit_txt))).click()

def set_pending_coursework():
    # Locate the Coursework Menu
    find_txt = "span.k-widget.k-dropdown.k-header[aria-owns='mCourseworkRangeId_1728_listbox']"
    longwait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, find_txt))).click()
    # Select Pending Coursework
    wait.until(EC.element_to_be_clickable((By.XPATH, "//li[text()='Pending Coursework']"))).click()

def find_xpath(x):
    xpath = '//*[@id="gridCoursework_1728"]/div[2]/table/tbody/tr[1]/td[' + str(x) + ']'
    result = driver.find_element(By.XPATH, xpath).text
    return(result)

def any_tasks():
# determine if there are any homework assignments
    try:
        find_xpath(1)
        proceed = True
    except:
        proceed = False

    return(proceed)

def mark_complete():
    # Locate the 3 dots and click them
    threedots = '//*[@id="gridCoursework_1728"]/div[2]/table/tbody/tr[1]/td[7]/img'
    wait.until(EC.element_to_be_clickable((By.XPATH, threedots))).click()

    # select 'Mark as Completed'
    completed_txt = '//*[@id="context-menu"]/li[3]/span'
    wait.until(EC.element_to_be_clickable((By.XPATH, completed_txt))).click()
    
def process_row():
    a_course  = find_xpath(2)
    a_title   = find_xpath(3)
    a_details = find_xpath(4)
    a_due     = find_xpath(6)

    task = {
        'course': a_course,
        'title': a_title,
        'details': a_details,
        'due': a_due
    }

    return(task)

def ok(method=1):
    # wrap the click_ok() function to catch errors
    result = True
    try:
        if method==1:
            # click ok button on popup window
            wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'btnOK'))).click()
        
        elif method==2:
            from selenium.webdriver.common.action_chains import ActionChains
            # Locate the OK button by its class name
            button = driver.find_element(By.CLASS_NAME, 'btnOK')
            driver.implicitly_wait(10)
            ActionChains(driver).move_to_element(button).click(button).perform()
        
        elif method==3:
            button = driver.find_element(By.CLASS_NAME, 'btnOK')

            # Scroll to the OK button
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

        elif method==4:
            button = driver.find_element(By.CLASS_NAME, 'btnOK k-button ')
            ## Scroll to the OK button
            #driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

        elif method==5:
            button = driver.find_element(By.CLASS_NAME, 'btnOK k-button ')

            relative = '//*[@id="top"]/div[29]/div[2]/button'
            absolute = '/html/body/div[29]/div[2]/button'
            button = driver.find_element(By.XPATH, absolute)

            ## Scroll to the OK button
            #driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

    except:
        # input("Click OK then press Enter to continue...")
        print("OK button was not clicked")
        result = False

    return result 

def posix_date(due):
    def click_time(entry):
        result = calendar.timegm(time.strptime(entry + ' 12:15:00', 
                                           '%m-%d-%Y %H:%M:%S')) * 1000
        return result 
    
    if (due == 'Today'):
        due = date.today().strftime('%m-%d-%Y')
    elif (due == 'Tomorrow'):
        due = (date.today() + timedelta(days=1)).strftime('%m-%d-%Y')

    start_date = click_time(date.today().strftime('%m-%d-%Y'))
    due_date = click_time(due)
    
    result = {"start": start_date, "due": due_date}
    return result 

def create_clickup_task(task):

    # Dictionary to convert from course id to tags
    c2tag = {
        '9PERFIN 5759/02': 'personal finance',
        '11ENG 3150/01': 'english',
        '11ALG2H 3351/01': 'algebra 2',
        '11USHIST 3450/01': 'us history',
        '11SPN3H 3551/01': 'spanish'
    }
    dates = posix_date(task['due'])
    
    task_data = {
        "name": task['title'],
        "description": task['details'],
        "status": "BACKLOG",
        "priority": 3,
        "due_date": dates['due'],
        "tags": [c2tag[task['course']]]
    }

    # Create the task
    response = requests.post(tasks_url, headers=headers, json=task_data)

    # Check the response
    if response.status_code == 200:
        print("Task created successfully!")
        task_info = response.json()
        print("Task ID:", task_info["id"])
        success = True
    else:
        print("Error creating task. Status code:", response.status_code)
        print("Response:", response.text)
        success = False

    return success 

def do_task():
    task = process_row()
    if create_clickup_task(task):
        mark_complete()
        worked = ok(3)
        if not worked:
            input("Click OK then press Enter to continue...")
        succeed = True
    else:
        print("Error creating clickup task")
        succeed = False

    return succeed 

In [2]:
# initialize assignments
login()
set_pending_coursework()

In [3]:
while any_tasks():
    if not do_task():
        break
# Close the WebDriver

Task created successfully!
Task ID: 8685mkw0n
OK button was not clicked


Click OK then press Enter to continue... 


Task created successfully!
Task ID: 8685mkw0r
OK button was not clicked


Click OK then press Enter to continue... 


Task created successfully!
Task ID: 8685mkw0t
OK button was not clicked


...

Click OK then press Enter to continue... 


Task created successfully!
Task ID: 8685mkw60
OK button was not clicked


Click OK then press Enter to continue... 


Task created successfully!
Task ID: 8685mkw6e
OK button was not clicked


Click OK then press Enter to continue... 


In [ ]:
do_task()

In [ ]:
driver.quit()

In [ ]:
def ok(method=1):
    # wrap the click_ok() function to catch errors
    try:
        if method==1:
            # click ok button on popup window
            wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'btnOK'))).click()
        
        elif method==2:
            from selenium.webdriver.common.action_chains import ActionChains
            button = driver.find_element(By.XPATH, "//button[text()='OK']")
            ActionChains(driver).move_to_element(button).click().perform()
        
        elif method==3:
            button = driver.find_element(By.CLASS_NAME, 'btnOK')

            # Scroll to the OK button
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

        elif method==4:
            button = driver.find_element(By.CLASS_NAME, 'btnOK k-button ')
            ## Scroll to the OK button
            #driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

        elif method==5:
            button = driver.find_element(By.CLASS_NAME, 'btnOK k-button ')

            relative = '//*[@id="top"]/div[29]/div[2]/button'
            absolute = '/html/body/div[29]/div[2]/button'
            button = driver.find_element(By.XPATH, absolute)

            ## Scroll to the OK button
            #driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

    except:
        input("Click OK then press Enter to continue...")


### Tasks to finish

1. Figure out how to add Ben's user id number